In [169]:
######## CODE ######### 
#Import csv file reader and math package
import csv
import math
 
# Open csv file 
with open('movie_rating_trainer_data.csv', 'rt') as f:
    file = list(csv.reader(f))
    
    
    #Retrieve and format data from csv file
    def retrieveFormatData(csv_file,no_of_parameters):
        #get list of movies
        movies = csv_file[0];
        movies = movies[1:-1];

        #Get movies unrated by user to recommend
        movies_to_recommend = movies[no_of_parameters:]

        #get training data - list of users and their ratings 
        users_array = file[1:];
        user_profiles = []
        temp_movie_list = []
        for x in range(len(users_array)):
            temp_ratings = []
            for y in range(len(movies_to_recommend)):
                temp_ratings.append((movies_to_recommend[y],int(users_array[x][y+no_of_parameters+1])))
            temp_movie_list.append(temp_ratings)

        #Format user infromation into the following format: [Name, CommonMoviesWithNewUser, MoviesToRecommend]
        #We are assuming there is a way for the system to identify what movies have and havent been watched by the new user 
        for user in range(len(users_array)):
            user_profiles.append([users_array[user][0],
                                  (int(users_array[user][1]),
                                   int(users_array[user][2]),
                                   int(users_array[user][3]),
                                   int(users_array[user][4]),
                                   int(users_array[user][5])),
                                    temp_movie_list[user]]) 
    
    
    #Function to get euclidean distance
    def euclideanDistance(instance1, instance2, length):
        distance = 0
        for x in range(length):
            distance += pow((instance1[x] - instance2[x]), 2)
        return math.sqrt(distance)
    
    
    #Function that takes a set of training users, a new user and a value for k to generate a list of k nearest users
    def getKNN(trainer_users,new,k):
        temp_distance_list = [];
        #Get list of euclidean distances to new user
        for user in trainer_users:
            dst = euclideanDistance(user[1], new[1], len(new[1]))
            temp_distance_list.append((user[0],dst,user[2]))
        #get k closest distances  
        temp_distance_list.sort(key=lambda tup: tup[1])  
        nearest_neighbours = temp_distance_list[0:k]
        return (nearest_neighbours)
    
    
    #Function that takes a list of k nearest neighbour users and finds 2 movie recommendations 
    def getRecMovies(knn,user_profiles,k):
        #Average ratings for the 5 possible recommendations
        avgs = [0] * len(knn[0][2])
        #For each neighbour find average ratins for possible reccommentations
        
        for n in knn:
            for x in range(len(n[2])):
                avgs[x] = avgs[x] + n[2][x][1]
        
        recommendations = []
        for x in range(len(movies_to_recommend)):
            recommendations.append((movies_to_recommend[x], avgs[x]/k))
        
        #Return 2 top liked movies by nearest neighbour users 
        recommendations.sort(key=lambda tup: tup[1], reverse=True)  
        recommendations = recommendations[0:2]

        #Print Recommendations
        print('1:' + recommendations[0][0])
        print('2:' + recommendations[1][0])
        

In [170]:
#Enter new user's details, ratings and a value for k (Between 1 & 5)
Name = 'Stephan'
DespicableMe = 4
InAndOut = 7
PrettyInPink = 0
StarWars = 10
Alien = 8
K = 3

retrieveFormatData(file,5)
#Create new user profile
new_user = [Name,(DespicableMe,InAndOut,PrettyInPink,StarWars,Alien)]  

#Get nearest neighbours
nearest_neighbours = getKNN(user_profiles,new_user,K)
print("User's nearest neighbours:")
print(nearest_neighbours)
print()
print("New user is recommended the following 2 movies")
getRecMovies(nearest_neighbours,user_profiles,K)

User's nearest neighbours:
[('Brian', 4.47213595499958, [('Mega Mind ', 2), ('Pretty Woman', 2), ('Breakfast Club', 3), ('Die Hard', 8), ('The Terminator', 9)]), ('Anna', 7.0710678118654755, [('Mega Mind ', 6), ('Pretty Woman', 3), ('Breakfast Club', 2), ('Die Hard', 7), ('The Terminator', 8)]), ('David', 7.0710678118654755, [('Mega Mind ', 3), ('Pretty Woman', 2), ('Breakfast Club', 3), ('Die Hard', 10), ('The Terminator', 10)])]

New user is recommended the following 2 movies
1:The Terminator
2:Die Hard
